In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
ratings = pd.read_csv('/kaggle/input/amazon-product-reviews/ratings_Electronics (1).csv')
ratings_df = ratings.copy()

In [ ]:
# Let's rename the columns 
ratings_df.columns = ['userId', 'productId', 'Rating', 'timestamp']
ratings_df

### Weighted recommendation

In [ ]:
total_votes = pd.DataFrame(ratings_df.groupby('productId')['Rating'].count().reset_index())
total_votes.columns = ['productId', 'vote_count']
total_votes = total_votes.merge(ratings_df, on = "productId")

mean_rating = total_votes.groupby('productId')['Rating'].mean().reset_index()
mean_rating = mean_rating.merge(total_votes, on = "productId")
mean_rating.columns = ['productId', 'avg rating', 'vote_count', 'userId', 'rating', 'timestamp']

In [ ]:
mean_rating.shape

In [ ]:
v = mean_rating['vote_count']
m = mean_rating['vote_count'].quantile(0.95)
R = mean_rating['avg rating']
C = mean_rating['rating'].mean()

mean_rating = mean_rating[mean_rating['vote_count'] >= m]

In [ ]:
mean_rating.shape

In [ ]:
recommended_products = mean_rating.drop(['userId', 'timestamp', 'rating'], axis = 1)
recommended_products.drop_duplicates(inplace = True)

In [ ]:
def recommendation(x):
    v = x['vote_count']
    R = x['avg rating']
    return (v/(v+m) * R) + (m/(m+v) * C)
recommended_products['score'] = recommended_products.apply(recommendation, axis = 1)

In [ ]:
recommended_products.sort_values(by ="score", ascending = False, inplace = True)

In [ ]:
recommended_products

### Collaborative Filtering 

In [ ]:
df_ratings = ratings.copy()

In [ ]:
df_ratings.columns = ['userId', 'productId', 'rating', 'timestamp']
df_ratings.drop('timestamp', axis = 1, inplace = True)

In [ ]:
df_ratings = df_ratings.head(10000)

In [ ]:
products_matrix = df_ratings.pivot_table(index = "productId", columns = "userId", values = "rating").fillna(0)

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = "brute")
products_sparse = csr_matrix(products_matrix.values)
model_knn.fit(products_sparse)

In [ ]:
def get_recommendations(prod_id):
    distances, indices = model_knn.kneighbors(products_matrix.loc[prod_id].values.reshape(1, -1), n_neighbors = 6)
    for i in range(0, len(distances.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(products_matrix.index[i]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, products_matrix.index[indices.flatten()[i]], distances.flatten()[i]))

In [ ]:
get_recommendations('B00000J1EP')